In [1]:
import warnings
import sys
import pathlib
import torch 
import numpy as np
import random
import optuna
from IPython.display import HTML
warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import pathlib
import warnings
warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
import torch 
import numpy as np
import random
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
import pandas as pd
from loaders.hrrr import discrete_scalar_field, discrete_vector_field
from fields.vector_field import ContinuousVectorField
def run_experiment(date: str, level: int):
    dsf = discrete_scalar_field(date, hours=4, level=f"{level} mb", extent=(-92, -74, 26, 40))
    cvf = ContinuousVectorField()
    cvf.train(dsf, epochs=100, sample_size=2000)
    dvf = discrete_vector_field(date, hours=4, level=f"{level} mb", extent=(-92, -74, 26, 40))
    return {
        "date": date,
        "level": level,
        "$\sigma^2$": float(cvf.sigma2),
        "$l_0$": float(cvf.l0),
        "$l_1$": float(cvf.l1),
        "$l_2$": float(cvf.l2),
        "RMS": float(dvf.RMS()),
        "RMSE":   float(cvf.RMSE(dvf))
    }

# 2. specify your dates, levels, and extent once
dates  = ["2024-03-15", "2024-06-21", "2024-09-18", "2024-12-25"]
levels = [500, 700]

# 3. collect results
results = []
for date in dates:
    for lvl in levels:
        results.append(run_experiment(date, lvl))

# 4. build DataFrame and export LaTeX
df = pd.DataFrame(results)
latex = df.to_latex(
    index=False,
    float_format="%.3f",
    caption="Estimated coveriance parameters and RMSE by date and pressure level",
    label="tab:params_rmse"
)
out = pathlib.Path(__file__).parent / "table.tex"
with open(out, "w") as f:
    f.write(latex)
print(f"Wrote LaTeX table to {out}")

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Mar-15 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
👨🏻‍🏭 Created directory: [/home/yf297/data/hrrr/20240315]
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Mar-15 00:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Mar-15 00:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Mar-15 00:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Mar-15 00:00 UTC F04 ┊ GRIB2 @ aws ┊ IDX @ aws
Epoch 1/100 — Avg NLL: 0.8847
Epoch 2/100 — Avg NLL: 0.8328
Epoch 3/100 — Avg NLL: 0.7973
Epoch 4/100 — Avg NLL: 0.7559
Epoch 5/100 — Avg NLL: 0.7185
Epoch 6/100 — Avg NLL: 0.6842
Epoch 7/100 — Avg NLL: 0.6445
Epoch 8/100 — Avg NLL: 0.6142
Epoch 9/100 — Avg NLL: 0.5749
Epoch 10/100 — Avg NLL: 0.5475
Epoch 11/100 — Avg NLL: 0.5037
Epoch 12/100 — Avg NLL: 0.4738
Epoch 13/100 — Avg NLL: 0.4454
Epoch 14/100 — Avg NLL: 0.4182
Epoch 15/100 — Avg NLL: 0.3744
Epoch 16/100 — Avg NLL: 0.3467
Epoch 17/10

KeyboardInterrupt: 